<a href="https://colab.research.google.com/github/JTStephens18/XCube-Paper-Implementation/blob/main/XCube_Paper_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Helpful Resources / References

- XCube Paper: https://research.nvidia.com/labs/toronto-ai/xcube/
- VAE refresher: https://www.youtube.com/watch?v=H2XgdND0DV4
- Amazing explanation of the math of diffusion models: https://www.youtube.com/watch?v=HoKDTa5jHvg
- Part 1 of the following talk was very helpful in checking my understanding of the DDPMs: https://www.youtube.com/watch?v=cS6JQpEY9cs
- https://github.com/Stability-AI/stablediffusion/blob/main/ldm/modules/diffusionmodules/util.py
- V-Parameterization trick: https://arxiv.org/pdf/2202.00512.pdf
- Explanation of the straight through estimator: https://hassanaskary.medium.com/intuitive-explanation-of-straight-through-estimators-with-pytorch-implementation-71d99d25d9d0#:~:text=A%20straight%2Dthrough%20estimator%20is,function%20was%20an%20identity%20function.

# VAE

## Preproccessing

### Process Binvox Models

In [ ]:
#  Copyright (C) 2012 Daniel Maturana
#  This file is part of binvox-rw-py.
#
#  binvox-rw-py is free software: you can redistribute it and/or modify
#  it under the terms of the GNU General Public License as published by
#  the Free Software Foundation, either version 3 of the License, or
#  (at your option) any later version.
#
#  binvox-rw-py is distributed in the hope that it will be useful,
#  but WITHOUT ANY WARRANTY; without even the implied warranty of
#  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#  GNU General Public License for more details.
#
#  You should have received a copy of the GNU General Public License
#  along with binvox-rw-py. If not, see <http://www.gnu.org/licenses/>.
#

"""
Binvox to Numpy and back.


>>> import numpy as np
>>> import binvox_rw
>>> with open('chair.binvox', 'rb') as f:
...     m1 = binvox_rw.read_as_3d_array(f)
...
>>> m1.dims
[32, 32, 32]
>>> m1.scale
41.133000000000003
>>> m1.translate
[0.0, 0.0, 0.0]
>>> with open('chair_out.binvox', 'wb') as f:
...     m1.write(f)
...
>>> with open('chair_out.binvox', 'rb') as f:
...     m2 = binvox_rw.read_as_3d_array(f)
...
>>> m1.dims==m2.dims
True
>>> m1.scale==m2.scale
True
>>> m1.translate==m2.translate
True
>>> np.all(m1.data==m2.data)
True

>>> with open('chair.binvox', 'rb') as f:
...     md = binvox_rw.read_as_3d_array(f)
...
>>> with open('chair.binvox', 'rb') as f:
...     ms = binvox_rw.read_as_coord_array(f)
...
>>> data_ds = binvox_rw.dense_to_sparse(md.data)
>>> data_sd = binvox_rw.sparse_to_dense(ms.data, 32)
>>> np.all(data_sd==md.data)
True
>>> # the ordering of elements returned by numpy.nonzero changes with axis
>>> # ordering, so to compare for equality we first lexically sort the voxels.
>>> np.all(ms.data[:, np.lexsort(ms.data)] == data_ds[:, np.lexsort(data_ds)])
True
"""

import numpy as np

class Voxels(object):
    """ Holds a binvox model.
    data is either a three-dimensional numpy boolean array (dense representation)
    or a two-dimensional numpy float array (coordinate representation).

    dims, translate and scale are the model metadata.

    dims are the voxel dimensions, e.g. [32, 32, 32] for a 32x32x32 model.

    scale and translate relate the voxels to the original model coordinates.

    To translate voxel coordinates i, j, k to original coordinates x, y, z:

    x_n = (i+.5)/dims[0]
    y_n = (j+.5)/dims[1]
    z_n = (k+.5)/dims[2]
    x = scale*x_n + translate[0]
    y = scale*y_n + translate[1]
    z = scale*z_n + translate[2]

    """

    def __init__(self, data, dims, translate, scale, axis_order):
        self.data = data
        self.dims = dims
        self.translate = translate
        self.scale = scale
        assert (axis_order in ('xzy', 'xyz'))
        self.axis_order = axis_order

    def clone(self):
        data = self.data.copy()
        dims = self.dims[:]
        translate = self.translate[:]
        return Voxels(data, dims, translate, self.scale, self.axis_order)

    def write(self, fp):
        write(self, fp)

def read_header(fp):
    """ Read binvox header. Mostly meant for internal use.
    """
    line = fp.readline().strip()
    if not line.startswith(b'#binvox'):
        raise IOError('Not a binvox file')
    dims = list(map(int, fp.readline().strip().split(b' ')[1:]))
    translate = list(map(float, fp.readline().strip().split(b' ')[1:]))
    scale = list(map(float, fp.readline().strip().split(b' ')[1:]))[0]
    line = fp.readline()
    return dims, translate, scale

def read_as_3d_array(fp, fix_coords=True):
    """ Read binary binvox format as array.

    Returns the model with accompanying metadata.

    Voxels are stored in a three-dimensional numpy array, which is simple and
    direct, but may use a lot of memory for large models. (Storage requirements
    are 8*(d^3) bytes, where d is the dimensions of the binvox model. Numpy
    boolean arrays use a byte per element).

    Doesn't do any checks on input except for the '#binvox' line.
    """
    dims, translate, scale = read_header(fp)
    raw_data = np.frombuffer(fp.read(), dtype=np.uint8)
    # if just using reshape() on the raw data:
    # indexing the array as array[i,j,k], the indices map into the
    # coords as:
    # i -> x
    # j -> z
    # k -> y
    # if fix_coords is true, then data is rearranged so that
    # mapping is
    # i -> x
    # j -> y
    # k -> z
    values, counts = raw_data[::2], raw_data[1::2]
    data = np.repeat(values, counts).astype(np.bool)
    data = data.reshape(dims)
    if fix_coords:
        # xzy to xyz TODO the right thing
        data = np.transpose(data, (0, 2, 1))
        axis_order = 'xyz'
    else:
        axis_order = 'xzy'
    return Voxels(data, dims, translate, scale, axis_order)

def read_as_coord_array(fp, fix_coords=True):
    """ Read binary binvox format as coordinates.

    Returns binvox model with voxels in a "coordinate" representation, i.e.  an
    3 x N array where N is the number of nonzero voxels. Each column
    corresponds to a nonzero voxel and the 3 rows are the (x, z, y) coordinates
    of the voxel.  (The odd ordering is due to the way binvox format lays out
    data).  Note that coordinates refer to the binvox voxels, without any
    scaling or translation.

    Use this to save memory if your model is very sparse (mostly empty).

    Doesn't do any checks on input except for the '#binvox' line.
    """
    dims, translate, scale = read_header(fp)
    raw_data = np.frombuffer(fp.read(), dtype=np.uint8)

    values, counts = raw_data[::2], raw_data[1::2]

    sz = np.prod(dims)
    index, end_index = 0, 0
    end_indices = np.cumsum(counts)
    indices = np.concatenate(([0], end_indices[:-1])).astype(end_indices.dtype)

    values = values.astype(np.bool)
    indices = indices[values]
    end_indices = end_indices[values]

    nz_voxels = []
    for index, end_index in zip(indices, end_indices):
        nz_voxels.extend(range(index, end_index))
    nz_voxels = np.array(nz_voxels)
    # TODO are these dims correct?
    # according to docs,
    # index = x * wxh + z * width + y; // wxh = width * height = d * d

    x = nz_voxels / (dims[0]*dims[1])
    zwpy = nz_voxels % (dims[0]*dims[1]) # z*w + y
    z = zwpy / dims[0]
    y = zwpy % dims[0]
    if fix_coords:
        data = np.vstack((x, y, z))
        axis_order = 'xyz'
    else:
        data = np.vstack((x, z, y))
        axis_order = 'xzy'

    #return Voxels(data, dims, translate, scale, axis_order)
    return Voxels(np.ascontiguousarray(data), dims, translate, scale, axis_order)

def dense_to_sparse(voxel_data, dtype=np.int):
    """ From dense representation to sparse (coordinate) representation.
    No coordinate reordering.
    """
    if voxel_data.ndim!=3:
        raise ValueError('voxel_data is wrong shape; should be 3D array.')
    return np.asarray(np.nonzero(voxel_data), dtype)

def sparse_to_dense(voxel_data, dims, dtype=np.bool):
    if voxel_data.ndim!=2 or voxel_data.shape[0]!=3:
        raise ValueError('voxel_data is wrong shape; should be 3xN array.')
    if np.isscalar(dims):
        dims = [dims]*3
    dims = np.atleast_2d(dims).T
    # truncate to integers
    xyz = voxel_data.astype(np.int)
    # discard voxels that fall outside dims
    valid_ix = ~np.any((xyz < 0) | (xyz >= dims), 0)
    xyz = xyz[:,valid_ix]
    out = np.zeros(dims.flatten(), dtype=dtype)
    out[tuple(xyz)] = True
    return out

#def get_linear_index(x, y, z, dims):
    #""" Assuming xzy order. (y increasing fastest.
    #TODO ensure this is right when dims are not all same
    #"""
    #return x*(dims[1]*dims[2]) + z*dims[1] + y

def write(voxel_model, fp):
    """ Write binary binvox format.

    Note that when saving a model in sparse (coordinate) format, it is first
    converted to dense format.

    Doesn't check if the model is 'sane'.

    """
    if voxel_model.data.ndim==2:
        # TODO avoid conversion to dense
        dense_voxel_data = sparse_to_dense(voxel_model.data, voxel_model.dims)
    else:
        dense_voxel_data = voxel_model.data

    fp.write('#binvox 1\n')
    fp.write('dim '+' '.join(map(str, voxel_model.dims))+'\n')
    fp.write('translate '+' '.join(map(str, voxel_model.translate))+'\n')
    fp.write('scale '+str(voxel_model.scale)+'\n')
    fp.write('data\n')
    if not voxel_model.axis_order in ('xzy', 'xyz'):
        raise ValueError('Unsupported voxel model axis order')

    if voxel_model.axis_order=='xzy':
        voxels_flat = dense_voxel_data.flatten()
    elif voxel_model.axis_order=='xyz':
        voxels_flat = np.transpose(dense_voxel_data, (0, 2, 1)).flatten()

    # keep a sort of state machine for writing run length encoding
    state = voxels_flat[0]
    ctr = 0
    for c in voxels_flat:
        if c==state:
            ctr += 1
            # if ctr hits max, dump
            if ctr==255:
                fp.write(chr(state))
                fp.write(chr(ctr))
                ctr = 0
        else:
            # if switch state, dump
            fp.write(chr(state))
            fp.write(chr(ctr))
            state = c
            ctr = 1
    # flush out remainders
    if ctr > 0:
        fp.write(chr(state))
        fp.write(chr(ctr))


    import doctest
    doctest.testmod()

### Helper Functions

In [ ]:
def calcNormals(voxelGrid):
  '''
    Calculating normal vectors for each point using central differences method
  '''
  shape = voxelGrid.shape

  dx = (torch.roll(voxelGrid, shifts=(-1, 0, 0), dims=(2, 3, 4)) - torch.roll(voxelGrid, shifts=(1, 0, 0), dims=(2, 3, 4))) / 2.0
  dy = (torch.roll(voxelGrid, shifts=(0, -1, 0), dims=(2, 3, 4)) - torch.roll(voxelGrid, shifts=(0, 1, 0), dims=(2, 3, 4))) / 2.0
  dz = (torch.roll(voxelGrid, shifts=(0, 0, -1), dims=(2, 3, 4)) - torch.roll(voxelGrid, shifts=(0, 0, 1), dims=(2, 3, 4))) / 2.0

  normal_vec = torch.stack([dx, dy, dz], dim=1)
  return normal_vec

In [ ]:
import plotly.graph_objects as go

def plotVoxels(input):
  '''
    Only show points where the value is greater than 0.5
  '''
  threshold = 0.5
  idx = 0
  channels = 0
  input = input.cpu()
  x, y, z = np.where(input[idx][channels] > threshold)

  fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode="markers", marker=dict(size=1))])

  fig.update_layout(scene=dict(xaxis_title='X', yaxis_title="Y", zaxis_title="Z"))

  fig.show()

### Data Loader

In [ ]:
"""
  Splits the data into a training (80%) and validation (20%) set
"""
filepath = "./binvoxModels_x64"

partition = {"train": [], "validation": []}
labels = {}

binvoxFiles = os.listdir(filepath)
splitSize = int(len(binvoxFiles) * 0.2)
randomIdx = random.sample(range(len(binvoxFiles)), splitSize)

for i, binvox in enumerate(binvoxFiles):
  if(binvox == ".ipynb_checkpoints"):
    print(i, binvox)
    continue;

  if(i in randomIdx):
    labels[binvox.split(".")[0]] = len(partition["validation"])
    partition["validation"].append(binvox)
  else:
    labels[binvox.split(".")[0]] = len(partition["train"])
    partition["train"].append(binvox)

In [ ]:
class VoxelDataset(Dataset):
  def __init__(self, data):
    self.samples = len(data)
    self.data = data

  def __len__(self):
    return self.samples

  def __getitem__(self, index):
    sample = self.data[index]
    return sample

In [ ]:
b_size = 16
training_set = VoxelDataset(partition["train"])
train_loader = torch.utils.data.DataLoader(training_set, batch_size=b_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Encoder

In [ ]:
class Encoder(nn.Module):
  def __init__(self, device):
    super(Encoder, self).__init__()

    self.conv_layer_1 = self._conv_layer_set(1, 64, 2, 1, 0, maxPool=True)
    self.conv_layer_2 = self._gcr_set(64, 64, 128, 2, 1, 0, maxPool=True)
    self.conv_layer_3 = self._gcr_set(128, 128, 256, 2, 1, 0, maxPool=True)
    self.sigmoid = nn.Sigmoid()
    self.device = device


  def _conv_layer_set(self, in_c, out_c, kernel_size, stride, padding, maxPool):
    if(maxPool == True):
      conv_layer = nn.Sequential(
          nn.Conv3d(in_c, out_c, kernel_size, stride, padding),
          nn.MaxPool3d(1, kernel_size)
      )
    else:
      conv_layer = nn.Sequential(
          nn.Conv3d(in_c, out_c, kernel_size, stride, padding)
      )
    return conv_layer


  def _gcr_set(self, num_g, in_c, out_c, kernel_size, stride, padding, maxPool):
    if(maxPool==True):
      GCR_layer = nn.Sequential(
        nn.GroupNorm(num_g, in_c),
        nn.Conv3d(in_c, out_c, kernel_size, stride, padding),
        nn.ReLU(),
        nn.MaxPool3d(1, kernel_size)
      )
    else:
      GCR_layer = nn.Sequential(
        nn.GroupNorm(num_g, in_c),
        nn.Conv3d(in_c, out_c, kernel_size, stride, padding),
      )
    return GCR_layer


  def forward(self, x):
    bn = self.conv_layer_1(x)
    bn = self.conv_layer_2(bn)
    bn = self.conv_layer_3(bn)
    return bn

## Decoder

In [ ]:
class Decoder(nn.Module):
  def __init__(self, device):
    super(Decoder, self).__init__()

    self.conv_transpose_1 = self._gcr_transpose_set(256, 256, 128, 4, 2, 1)
    self.conv_transpose_2 = self._gcr_transpose_set(128, 128, 64, 4, 2, 1)
    self.conv_transpose_3 = self._gcr_transpose_set(64, 64, 1, 4, 2, 1)

  def _gcr_transpose_set(self, num_g, in_c, out_c, kernel_size, stride, padding):
    gcr_layer = nn.Sequential(
        nn.GroupNorm(num_g, in_c),
        nn.ConvTranspose3d(in_c, out_c, kernel_size, stride, padding),
        nn.ReLU(),
    )
    return gcr_layer


  def forward(self, x):
    z = self.conv_transpose_1(x)
    z = self.conv_transpose_2(z)
    z = self.conv_transpose_3(z)
    output = z
    return output

## Straight Through Estimator

In [ ]:
class STEFunction(torch.autograd.Function):
  @staticmethod
  def forward(ctx, input):
    return (input > 0.5).float()

  @staticmethod
  def backward(ctx, grad_output):
    return F.hardtanh(grad_output)

In [ ]:
class StraightThroughEstimator(nn.Module):
  def __init__(self):
    super(StraightThroughEstimator, self).__init__()

  def forward(self, x):
    x= STEFunction.apply(x)
    return x

## VAE

In [ ]:
class VAE(nn.Module):
  def __init__(self, Encoder, Decoder, STE, device):
    super(VAE, self).__init__()

    self.Encoder = Encoder
    self.Decoder = Decoder
    self.device = device
    self.flatten = nn.Flatten()
    self.z_mean = nn.Linear(131072, 16)
    self.z_log_var = nn.Linear(131072, 16)
    self.reshape = nn.Sequential(
        nn.Linear(16, 131072)
    )
    self.STE = STE

  def reparameterize(self, z_mu, z_log_var):
    eps = torch.randn_like(z_log_var)
    z = z_mu + eps * torch.exp(z_log_var*0.5)
    return z

  def forward(self, x):
    bn = self.Encoder(x)

    latent_vec = self.flatten(bn)
    z_mean = self.z_mean(latent_vec)
    z_log_var = self.z_log_var(latent_vec)
    z_log_var = torch.clamp(z_log_var, min=-100.0, max=100.0)
    z = self.reparameterize(z_mean, z_log_var)
    z = self.reshape(z)
    z = z.view(x.shape[0], 256, 8, 8, 8)

    output = self.Decoder(z)

    output = self.STE(output)

    return output, z_mean, z_log_var

In [ ]:
vae = VAE(Encoder(device), Decoder(device), StraightThroughEstimator(), device).to(device)
vae.train()

## Training Loop

In [ ]:
learning_rate = 0.0001
optimizer = torch.optim.Adam(vae.parameters(), learning_rate)

In [ ]:
epochs = 100
step = 0
loss = -1
train_loader_len = len(train_loader)
for epoch in range(epochs):
  print("***************** Epoch: ", epoch, "***********************")
  checkpoint = {
      "epoch": epoch,
      "model_state_dict": vae.state_dict(),
      "optimizer_state_dict": optimizer.state_dict(),
      'loss': loss,
  }

  torch.save(checkpoint, 'VAECheckpoint.pth')
  ! cp VAECheckpoint.pth /path/to/save

  for idx, data in enumerate(train_loader):
    if(idx >= train_loader_len - 1):
      continue
    step += 1
    tempTensorList = []
    tempNormalsList = []
    idx+=1
    for item in range(len(data)):
      convertedModel = torch.load(f"./binvoxModels_x64/{data[item]}")
      tempTensorList.append(convertedModel)
      normals = torch.load(f'./binvoxNormals_x64/{data[item]}.pt')
      tempNormalsList.append(normals)
    inputTensor = torch.stack(tempTensorList, dim=0).to(device)
    normalsTensor = torch.stack(tempNormalsList, dim=0).to(device)
    outputTensor, z_mean, z_log_var = vae(inputTensor)

    # Calculate normals for predicted output
    predictedNormals = calcNormals(outputTensor)
    reshapedOriginalNormals = normalsTensor.view(normalsTensor.shape[0], normalsTensor.shape[1], -1)
    reshapedPredNormals = predictedNormals.view(predictedNormals.shape[0], predictedNormals.shape[1], -1)
    L2NormOriginal = torch.linalg.norm(reshapedOriginalNormals, dim=2)
    L2NormPred = torch.linalg.norm(reshapedPredNormals, dim=2)
    normalsWeight = 1.0
    normalsLoss = torch.abs(torch.sum(normalsWeight * (L2NormPred - L2NormOriginal)) / 48 )

    bceLoss = nn.BCELoss()
    bceResult = bceLoss(inputTensor, outputTensor)

    kl_weight = 0.0015
    kl_div = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - torch.exp(z_log_var), dim=1)
    kl_div = kl_div.mean()

    loss = (bceResult + normalsLoss) + (kl_weight * kl_div)

    optimizer.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()

    if step % 50 == 0:
      print("Epoch", epoch, "step", step, "Loss", loss.detach().item(), "BCE", bceResult.detach().item(), "KL_Div", (kl_weight*kl_div).detach().item())
      for name, param in vae.named_parameters():
        if param.grad is not None:
          print(name, torch.norm(param.grad).item())

# Diffusion Model

## Helper Functions

In [ ]:
def linear_beta_schedule(timesteps, start=0.0001, end=0.02):
  return torch.linspace(start, end, timesteps)

def get_index_from_list(vals, t, x_shape):
  """
  Returns a specific index t of a list of values 'vals'
  """
  batch_size = t.shape[0]
  out = vals.gather(-1, t.cpu())
  return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(device)

def forward_diffusion_sample(x_0, t, device):
  """
  Takes an image and a timestep and returns
  noisy version of the image at the timestep
  """

  noise = torch.randn_like(x_0)
  sqrt_alphas_cumprod_t = get_index_from_list(sqrt_alphas_cumprod, t, x_0.shape)
  sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
      sqrt_one_minus_alphas_cumprod, t, x_0.shape
  )
  # Mean + variance
  return sqrt_alphas_cumprod_t.to(device) * x_0.to(device) + sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device), noise.to(device)


# Define beta schedule
timesteps = 1000
betas = linear_beta_schedule(timesteps=timesteps)

# Pre-calcualte different terms for closed form
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1,0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

In [ ]:
def get_v_loss(x_0, noise, t):
  v = get_index_from_list(sqrt_alphas_cumprod, t, x_0.shape) * noise - get_index_from_list(sqrt_one_minus_alphas_cumprod, t, x_0.shape) * x_0
  return v

## Archictecture

In [ ]:
class Block(nn.Module):
  def __init__(self, in_c, out_c, time_emb_dim, up=False):
    super().__init__()

    self.time_mlp = nn.Linear(time_emb_dim, out_c).to(device)

    if up:
      self.conv_1 = nn.Conv3d(in_c, out_c, 3, padding=1).to(device)
      self.transform = nn.ConvTranspose3d(out_c, out_c, 1).to(device)
    else:
      self.conv_1 = nn.Conv3d(in_c, out_c, 3, padding=1).to(device)
      self.transform = nn.Conv3d(out_c, out_c, 4, 2, 1).to(device)
    self.conv_2 = nn.Conv3d(out_c, out_c, 3, padding=1).to(device)
    self.bnorm_1 = nn.BatchNorm3d(out_c).to(device)
    self.bnorm_2 = nn.BatchNorm3d(out_c).to(device)
    self.relu = nn.ReLU().to(device)

  def forward(self, x, t):
    out = self.bnorm_1(self.relu(self.conv_1(x)))
    # Time Embedding
    time_emb = self.relu(self.time_mlp(t))
    # Extend last 2 dims
    time_emb = time_emb[(...,) + (None,) * 3]
    # Add time embedding
    out = out + time_emb
    out = self.bnorm_2(self.relu(self.conv_2(out)))
    return self.transform(out)

In [ ]:
class SinusoidalPositionEmbeddings(nn.Module):
  def __init__(self, dim):
    super().__init__()
    self.dim = dim

  def forward(self, time):
    device = time.device
    half_dim = self.dim // 2
    embeddings = math.log(10000) / (half_dim - 1)
    embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
    embeddings = time[:, None] * embeddings[None, :]
    embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
    return embeddings

In [ ]:
class Unet(nn.Module):
  def __init__(self):
    super().__init__()
    in_c = 256
    channels = (256, 512, 1024, 2048)
    out_dim = 256
    time_emb_dim = 32

    self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim),
            nn.ReLU()
        ).to(device)

    self.conv_0 = nn.Conv3d(in_c, channels[0], 3, padding=1).to(device)
    self.down_1 = Block(channels[0], channels[1], time_emb_dim)
    self.down_2 = Block(channels[1], channels[2], time_emb_dim)
    self.down_3 = Block(channels[2], channels[2], time_emb_dim)

    self.up_conv_1 = nn.ConvTranspose3d(channels[2], channels[2], 4, 2, 1).to(device)
    self.up_conv_2 = nn.ConvTranspose3d(channels[2], channels[1], 4, 2, 1).to(device)
    self.up_conv_3 = nn.ConvTranspose3d(channels[1], channels[0], 4, 2, 1).to(device)

    self.dec_1 = Block(channels[3], channels[2], time_emb_dim, up=True)
    self.dec_2 = Block(channels[2], channels[1], time_emb_dim, up=True)
    self.dec_3 = Block(channels[1], channels[0], time_emb_dim, up=True)

  def forward(self, x, timestep):
    skip_connections = []
    t = self.time_mlp(timestep)

    x = self.conv_0(x)
    skip_connections.append(x)

    x = self.down_1(x, t)
    skip_connections.append(x)

    x = self.down_2(x, t)
    skip_connections.append(x)

    x = self.down_3(x, t)

    x = self.up_conv_1(x)
    skip_1 = skip_connections.pop()
    x = torch.cat((x, skip_1), dim=1)

    x = self.dec_1(x, t)

    x = self.up_conv_2(x)
    skip_2 = skip_connections.pop()
    x = torch.cat((x, skip_2), dim=1)

    x = self.dec_2(x, t)

    x = self.up_conv_3(x)
    skip_3 = skip_connections.pop()
    x = torch.cat((x, skip_3), dim=1)

    x = self.dec_3(x, t)
    return x

In [ ]:
model = Unet()

## Training Loop

In [ ]:
lr = 0.00005
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
encoder = vae.Encoder
epochs = 100
step = 0
loss = -1
timestep_inc = (timesteps / epochs)
for epoch in range(epochs):
  print("************** Epoch:", epoch, "******************")
  checkpoint = {
      "epoch": epoch,
      "model_state_dict": model.state_dict(),
      "optimizer_state_dict": optimizer.state_dict(),
      "loss": loss
  }
  torch.save(checkpoint, "DiffusionCheckpoint.pth")
  ! cp DiffusionCheckpoint.pth /path/to/savepoint

  for idx, data in enumerate(train_loader):
    if(idx >= len(train_loader) - 1):
      continue
    step += 1

    tempTensorList = []
    for item in range(len(data)):
      voxels = torch.load(f"./binvoxModels_x64/{data[item]}")
      tempTensorList.append(voxels)
    inputTensor = torch.stack(tempTensorList, dim=0).to(device)
    with torch.no_grad():
      encodedOutput = encoder(inputTensor)

    time = torch.tensor([timestep_inc * epoch]).type(torch.int64).to(device)
    noisy_voxels, noise = forward_diffusion_sample(encodedOutput, time, device)
    pred = model(noisy_voxels, time)

    mseLoss = nn.MSELoss()
    target = get_v_loss(encodedOutput, noise, time)
    loss = mseLoss(target, pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 50 == 0:
      print("Epoch", epoch, "loss", loss.detach().item(), "step", step)
      for name, param in model.named_parameters():
        if param.grad is not None:
          print(name, torch.norm(param.grad).item())